# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
from openai import OpenAI
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
import requests

In [4]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

In [3]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')


In [5]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2:1b'

In [6]:
def generate_questions_about_topic_gpt(topic, client=None):
    """
    Generate 3 questions about the given topic using GPT-4o mini.
    Returns a dict with key "questions" (list of 3 strings).
    """
    if client is None:
        client = OpenAI()
    system_prompt = (
        "You are an intelligent student trying to find out more about the topic you are studying. "
        "Your task is to generate exactly 3 thoughtful questions that would help you deepen your understanding. "
        "Respond only with valid JSON in this exact format: {\"questions\": [\"question 1\", \"question 2\", \"question 3\"]}."
    )
    user_prompt = f"The topic I am studying is: {topic}. Generate 3 questions about it."
    response = client.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        response_format={"type": "json_object"},
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [7]:
def tutor_answers_for_topic_gpt(topic, client=None):
    """
    Get 3 questions about the topic, then ask the LLM as a tutor to answer each question.
    Results stream back from OpenAI with a typewriter-style animation (markdown, max 400 tokens per answer).
    Returns Markdown so the cell output renders correctly (e.g. in Cursor/VS Code).
    """
    if client is None:
        client = OpenAI()
    data = generate_questions_about_topic_gpt(topic, client=client)
    questions = data.get("questions", [])
    tutor_system = (
        "You are a helpful tutor. Answer the student's question concisely and clearly. "
        "Keep your answer under 400 tokens. Respond in markdown format."
    )
    full_response = ""
    display_handle = display(Markdown(""), display_id=True)
    for i, q in enumerate(questions, 1):
        header = f"## Question {i}\n\n{q}\n\n### Answer\n\n"
        full_response += header
        update_display(Markdown(full_response), display_id=display_handle.display_id)
        stream = client.chat.completions.create(
            model=MODEL_GPT,
            messages=[
                {"role": "system", "content": tutor_system},
                {"role": "user", "content": q},
            ],
            max_tokens=400,
            stream=True,
        )
        for chunk in stream:
            delta = chunk.choices[0].delta.content if chunk.choices else None
            full_response += delta or ""
            update_display(Markdown(full_response), display_id=display_handle.display_id)
        full_response += "\n\n---\n\n"
    update_display(Markdown(full_response), display_id=display_handle.display_id)


In [8]:
# Example: get 3 questions about a topic and tutor answers (streaming typewriter effect).
# Function returns Markdown so the cell output renders correctly (e.g. in Cursor/VS Code).
tutor_answers_for_topic_gpt("sport")

## Question 1

What are the physical and mental benefits of participating in regular sports activities?

### Answer

Participating in regular sports activities offers a range of physical and mental benefits. Here’s a breakdown of these advantages:

### Physical Benefits

1. **Improved Cardiovascular Health**: Engaging in sports enhances heart health, increases stamina, and improves circulation, reducing the risk of heart disease.

2. **Enhanced Strength and Flexibility**: Regular activity strengthens muscles and bones, increases flexibility, and improves overall body composition.

3. **Weight Management**: Sports help to burn calories, which is beneficial for maintaining a healthy weight or losing weight.

4. **Strengthened Immune System**: Regular physical activity can boost the immune system, making the body more resistant to infections.

5. **Better Coordination and Balance**: Many sports improve coordination, balance, and agility, which are important for overall physical function and injury prevention.

6. **Increased Endurance**: Ongoing participation in sports can lead to increased endurance, allowing for more prolonged physical activity.

### Mental Benefits

1. **Stress Relief**: Physical activity increases the production of endorphins, which help reduce stress levels and improve mood.

2. **Enhanced Cognitive Function**: Regular engagement in sports can improve concentration, memory, and overall cognitive function due to better blood flow to the brain.

3. **Boosted Self-Esteem**: Achieving personal fitness goals or excelling in a sport can enhance self-confidence and self-worth.

4. **Social Interaction**: Team sports foster social connections, reducing feelings of loneliness and improving overall emotional well-being.

5. **Improved Sleep Quality**: Regular physical activity can help regulate sleep patterns, leading to more restful and restorative sleep.

6. **Mental Resilience**: Participating in sports often involves facing challenges and overcoming obstacles, which can build mental toughness and resilience.

### Conclusion

Incorporating regular sports activities into your routine can significantly enhance both your physical health and mental well-being. Whether through team sports or individual activities, the benefits extend across various

---

## Question 2

How do different cultures influence the popularity and style of various sports around the world?

### Answer

Different cultures significantly shape the popularity and style of various sports in several ways:

### 1. Historical Context
- **Tradition and History**: Many sports are tied to a region’s history. For instance, cricket in England has deep historical roots, while baseball is often associated with American culture.
- **Colonization**: Sports introduced during colonization, such as soccer in many African nations, have often become integral to national identity.

### 2. Social Factors
- **Community and Identity**: Sports often reflect community values. For example, sumo wrestling in Japan is linked to Shinto rituals and embodies Japanese culture.
- **Gender Roles**: In some cultures, gender can influence the types of sports pursued. In certain Middle Eastern countries, women's participation in sports like soccer may be limited compared to nations with more progressive views on gender equality.

### 3. Economic Factors
- **Investment and Infrastructure**: Countries with more resources can build better facilities and promote certain sports. For instance, the U.S. invests heavily in American football, leading to its popularity.
- **Sponsorships**: Sports that attract more sponsorship are often more promoted and popular in specific regions, like basketball in the U.S.

### 4. Media and Globalization
- **Media Exposure**: Sports gain popularity through media coverage. The rise of global sports leagues (like the NBA) helps popularize basketball worldwide.
- **Social Media Influence**: Platforms allow for rapid spread of cultural sports trends, making them popular beyond their original context.

### 5. Cultural Values
- **Physicality and Competition**: Cultures that value physical strength may lean toward combat sports (like wrestling in Mongolia), while cultures emphasizing teamwork may favor sports like basketball or soccer.
- **Celebration and Festival**: Some sports are linked to community festivals and celebrations (like the traditional game of Kabaddi in India), which influences their popularity.

### Conclusion
Cultural influences are multifaceted, impacting not

---

## Question 3

What role does technology play in enhancing performance and safety in sports today?

### Answer

Technology plays a crucial role in enhancing performance and safety in sports in several key areas:

### 1. **Performance Analysis**
- **Wearable Technology:** Devices like smartwatches and fitness trackers monitor athletes' metrics (heart rate, distance, speed) to provide real-time data, helping athletes and coaches make informed decisions to enhance performance.
- **Video Analysis:** High-speed cameras and motion capture systems analyze athletes' techniques and movements, allowing for detailed feedback and the ability to refine skills.

### 2. **Training and Preparation**
- **Virtual Reality (VR):** Used for immersive training simulations, VR can replicate competitive environments, helping athletes prepare mentally and physically without the risks associated with actual play.
- **Data Analytics:** Analyzing large datasets helps teams and athletes understand performance patterns, identify strengths and weaknesses, and tailor training programs specifically for individual athletes.

### 3. **Injury Prevention and Recovery**
- **Biomechanics:** Advanced motion analysis helps identify potential injury risks by assessing movement patterns, enabling coaches to modify techniques that may lead to injury.
- **Rehabilitation Technology:** Innovations such as robotic exoskeletons and physiotherapy tools aid in recovery, ensuring athletes can return to their sport safely and effectively.

### 4. **Equipment Innovation**
- **Smart Equipment:** Items like smart balls and sensor-embedded gear provide feedback on mechanics and performance, helping athletes improve technique.
- **Safety Gear:** Technological advancements in materials have led to lighter, more protective sports gear, such as helmets and pads, significantly reducing injury risks.

### 5. **Fan Engagement and Experience**
- **Interactive Technologies:** Augmented reality (AR) and apps enhance the fan experience during events, allowing for deeper engagement with the sport and its athletes, leading to increased support and revenue for teams.

### Conclusion
Overall, technology is a transformative force in sports, enhancing not just athletic performance but also significantly contributing to safety and injury prevention. By integrating the latest technological advancements, athletes and teams can

---



In [9]:
# --- Same 2 functions using Ollama (MODEL_LLAMA). Only client and MODEL differ from GPT versions. ---

def generate_questions_about_topic_ollama(topic, client=None):
    """
    Same as generate_questions_about_topic_gpt; only client (ollama) and model (MODEL_LLAMA) differ.
    """
    if client is None:
        client = ollama
    system_prompt = (
        "You are an intelligent student trying to find out more about the topic you are studying. "
        "Your task is to generate exactly 3 thoughtful questions that would help you deepen your understanding. "
        "Respond only with valid JSON in this exact format: {\"questions\": [\"question 1\", \"question 2\", \"question 3\"]}."
    )
    user_prompt = f"The topic I am studying is: {topic}. Generate 3 questions about it."
    response = client.chat.completions.create(
        model=MODEL_LLAMA,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        response_format={"type": "json_object"},
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
def tutor_answers_for_topic_ollama(topic, client=None):
    """
    Same as tutor_answers_for_topic_gpt; only client (ollama) and model (MODEL_LLAMA) differ.
    """
    if client is None:
        client = ollama
    data = generate_questions_about_topic_ollama(topic, client=client)
    questions = data.get("questions", [])
    tutor_system = (
        "You are a helpful tutor. Answer the student's question concisely and clearly. "
        "Keep your answer under 400 tokens. Respond in markdown format."
    )
    full_response = ""
    display_handle = display(Markdown(""), display_id=True)
    for i, q in enumerate(questions, 1):
        header = f"## Question {i}\n\n{q}\n\n### Answer\n\n"
        full_response += header
        update_display(Markdown(full_response), display_id=display_handle.display_id)
        response = client.chat.completions.create(
            model=MODEL_LLAMA,
            messages=[
                {"role": "system", "content": tutor_system},
                {"role": "user", "content": q},
            ],
            max_tokens=400,
        )
        answer = (response.choices[0].message.content or "").strip()
        full_response += answer + "\n\n---\n\n"
        update_display(Markdown(full_response), display_id=display_handle.display_id)
    update_display(Markdown(full_response), display_id=display_handle.display_id)


In [12]:
# Example: same flow using Ollama locally (llama3.2:1b). Requires Ollama running with model pulled.

tutor_answers_for_topic_ollama("getting rid of gambling addiction")

## Question 1

What are some effective strategies for identifying and addressing underlying factors that contribute to the development of a person's gambling addiction, such as psychological issues or trauma?

### Answer

# Defining Gambling Addiction

Gambling addiction is a complex condition with multiple contributing factors. Identifying underlying psychological issues and trauma can be crucial in developing an effective treatment plan.

## Psychological Factors

*   Cognitive distortions: distorted thinking patterns
*   Compulsive behaviors: repetitive gambling attempts
*   Emotional regulation difficulties: impulse control issues
*   Impairment of decision-making skills
*   Social pressures and family dynamics:

### Identifying underlying psychological factors through self-reflection

1.  Recognize and acknowledge personal thoughts, feelings, and experiences.
2.  Explore past traumas or negative life events that may have led to gambling addiction.

## Trauma-Related Factors

*   Post-traumatic stress disorder (PTSD)
*   Childhood trauma: neglect, abuse, or domestic violence
*   Adult trauma: marital disharmony or family conflicts
*   Loss of a loved one
*   Physical illness

### Addressing underlying trauma through therapy

1.  Collaborate with a mental health professional to create a supportive environment.
2.  Work through past traumas using therapeutic techniques such as CBT, psychodynamic therapy, or trauma-focused cognitive-behavioral therapy (TF-CBT).

## Integrated Treatment Approach

*   Combine psychological and trauma-related factors assessment to develop an effective treatment plan.
*   Engage in evidence-based therapies like DBT (Dialectical Behavior Therapy) and cognitive-behavioral therapy (CBT).
*   Incorporate family participation and support.

By addressing the underlying psychological issues and trauma, individuals can develop a more comprehensive understanding of their gambling addiction and work towards recovery.

---

## Question 2

How can healthcare providers support individuals in gaining insights into their motivations and thoughts associated with gambling while also providing evidence-based guidance on quitting?

### Answer

### Supporting Individuals in Gaining Insights into Gamblers' Motivations and Thoughts

Healthcare providers can effectively assist individuals in understanding their motivations and thoughts around gambling by:

*   **Creating a Safe Space for Dialogue**
    Providing a non-judgmental atmosphere where individuals feel comfortable sharing their concerns and emotions.
*   **Conducting Open-Ended Questions**
    Encouraging open-ended conversations to gather information about the individual's thought process, motivations, and behaviors related to gambling.

### Evidence-Based Guidance on Quitting Gambling

To provide effective evidence-based guidance, healthcare providers can:

*   **Identify Individualized Needs and Barriers to Quitting**
    Assessing the individual's specific needs, coping mechanisms, and environmental factors that may be hindering their ability to quit.
*   **Introduce Effective Coping Strategies**
    Supporting individuals in developing healthy coping techniques, such as stress management, relaxation, and social support.

### Combining Support with Evidence-Based Interventions

By combining support with evidence-based interventions, healthcare providers can:

1.  **Enhance Treatment Success Rates**
2.  **Improve Relapse Prevention**
3.  **Increase Overall Wellness**

Healthcare providers should emphasize that quitting gambling is a unique process for each individual and may require tailored approaches to address their specific needs and motivations.

### Additional Resources

For further guidance, healthcare providers can:

*   Consult with addiction specialists or mental health professionals who have experience working with gamblers
*   Utilize online resources and support groups
*   Collaborate with community organizations that specialize in supporting individuals struggling with gambling and addiction.

---

## Question 3

In what ways do sociocultural and environmental factors influence an individual's propensity to gamble and how can interventions aim to mitigate these effects?

### Answer

### Sociocultural Factors:

*   **Cultural Background**: Gamblers from different cultures may have varying attitudes towards chance, risk-taking, and financial dependence.
*   **Social Influence**: Peer pressure and social norms can significantly influence an individual's likelihood of engaging in gambling.
+   Interventions to mitigate this effect:
    +   Education and awareness campaigns promoting responsible gaming practices
    +   Encouraging support groups for individuals struggling with addiction

### Environmental Factors:

*   **Access to Gambling**: Easy access to casinos, arcades, or online platforms can increase the likelihood of engagement.
+   Interventions to mitigate this effect:
    +   Location-based restrictions on gambling venues (e.g., casino-free zones)
    +   Online platform ban or moderation measures

### Mitigating Interventions:

*   **Risk-Benefit Analysis**: Assessing an individual's risk tolerance and benefits from substance use can help set realistic expectations.
*   **Gradual Introduction to Problem Gaming**: Gradually introducing new habits, such as problem gaming, under close supervision can minimize escalation.
*   **Support Systems**: Encouraging individuals with addiction or risk-taking behavior to seek professional support, such as counseling or therapy.

---

